In [33]:
from sklearn.metrics import precision_recall_fscore_support

In [34]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, BertModel, AdamW
from collections.abc import Iterable
from torchtext import data
from transformers import BertModel
import torch.nn as nn
from transformers.optimization import *
import torch.optim as optim
import math
from torch.nn.utils.rnn import pad_sequence

In [35]:

from torch.utils.data.dataloader import DataLoader
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [36]:
import logging
import os
import sys
import argparse
import random

import numpy as np
import tqdm
import pickle
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
df_train = pd.read_csv("snli_1.0/snli_1.0_train.txt", sep="\t" )
df_dev = pd.read_csv("snli_1.0/snli_1.0_dev.txt", sep="\t")
df_test = pd.read_csv("snli_1.0/snli_1.0_test.txt", sep="\t")

In [ ]:
df_train[:10]

,gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,captionID,pairID,label1,label2,label3,label4,label5
0,neutral,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,3416050480.jpg#4,3416050480.jpg#4r1n,neutral,NaN,NaN,NaN,NaN
1,contradiction,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",3416050480.jpg#4,3416050480.jpg#4r1c,contradiction,NaN,NaN,NaN,NaN
2,entailment,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,"( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",3416050480.jpg#4,3416050480.jpg#4r1e,entailment,NaN,NaN,NaN,NaN
3,neutral,( Children ( ( ( smiling and ) waving ) ( at c...,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...,Children smiling and waving at camera,They are smiling at their parents,2267923837.jpg#2,2267923837.jpg#2r1n,neutral,NaN,NaN,NaN,NaN
4,entailment,( Children ( ( ( smiling and ) waving ) ( at c...,( There ( ( are children ) present ) ),(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...,Children smiling and waving at camera,There are children present,2267923837.jpg#2,2267923837.jpg#2r1e,entailment,NaN,NaN,NaN,NaN
5,contradiction,( Children ( ( ( smiling and ) waving ) ( at c...,( ( The kids ) ( are frowning ) ),(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (DT The) (NNS kids)) (VP (VBP are...,Children smiling and waving at camera,The kids are frowning,2267923837.jpg#2,2267923837.jpg#2r1c,contradiction,NaN,NaN,NaN,NaN
6,contradiction,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,( ( The boy ) ( ( ( skates down ) ( the sidewa...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ skate...,A boy is jumping on skateboard in the middle o...,The boy skates down the sidewalk.,3691670743.jpg#0,3691670743.jpg#0r1c,contradiction,NaN,NaN,NaN,NaN
7,entailment,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,( ( The boy ) ( ( does ( a ( skateboarding tri...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ does)...,A boy is jumping on skateboard in the middle o...,The boy does a skateboarding trick.,3691670743.jpg#0,3691670743.jpg#0r1e,entailment,NaN,NaN,NaN,NaN
8,neutral,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,( ( The boy ) ( ( is ( wearing ( safety equipm...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ is) (...,A boy is jumping on skateboard in the middle o...,The boy is wearing safety equipment.,3691670743.jpg#0,3691670743.jpg#0r1n,neutral,NaN,NaN,NaN,NaN
9,neutral,( ( An ( older man ) ) ( ( ( sits ( with ( ( h...,( ( An ( older man ) ) ( ( ( drinks ( his juic...,(ROOT (S (NP (DT An) (JJR older) (NN man)) (VP...,(ROOT (S (NP (DT An) (JJR older) (NN man)) (VP...,An older man sits with his orange juice at a s...,An older man drinks his juice as he waits for ...,4804607632.jpg#0,4804607632.jpg#0r1n,neutral,NaN,NaN,NaN,NaN


In [ ]:
df_train.drop(df_train.columns.difference(['gold_label','sentence1', 'sentence2']), axis=1, inplace=True) ## to drop the columns
df_test.drop(df_test.columns.difference(['gold_label','sentence1', 'sentence2']), axis=1, inplace=True) ## to drop the columns
df_dev.drop(df_dev.columns.difference(['gold_label','sentence1', 'sentence2']), axis=1, inplace=True) ## to drop the columns

In [ ]:
df_train[:10]

,gold_label,sentence1,sentence2
0,neutral,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.
1,contradiction,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette."
2,entailment,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse."
3,neutral,Children smiling and waving at camera,They are smiling at their parents
4,entailment,Children smiling and waving at camera,There are children present
5,contradiction,Children smiling and waving at camera,The kids are frowning
6,contradiction,A boy is jumping on skateboard in the middle o...,The boy skates down the sidewalk.
7,entailment,A boy is jumping on skateboard in the middle o...,The boy does a skateboarding trick.
8,neutral,A boy is jumping on skateboard in the middle o...,The boy is wearing safety equipment.
9,neutral,An older man sits with his orange juice at a s...,An older man drinks his juice as he waits for ...


In [ ]:
df_train = df_train[df_train['gold_label'] != '-']
df_dev = df_dev[df_dev['gold_label'] != '-']
df_test = df_test[df_test['gold_label'] != '-']

In [ ]:
df_train


,gold_label,sentence1,sentence2
0,neutral,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.
1,contradiction,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette."
2,entailment,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse."
3,neutral,Children smiling and waving at camera,They are smiling at their parents
4,entailment,Children smiling and waving at camera,There are children present
...,...,...,...
550147,contradiction,Four dirty and barefooted children.,four kids won awards for 'cleanest feet'
550148,neutral,Four dirty and barefooted children.,"four homeless children had their shoes stolen,..."
550149,neutral,A man is surfing in a bodysuit in beautiful bl...,A man in a bodysuit is competing in a surfing ...
550150,contradiction,A man is surfing in a bodysuit in beautiful bl...,A man in a business suit is heading to a board...


In [ ]:
# Analyzing the data
df_train.groupby('gold_label').count()

,sentence1,sentence2
gold_label,,
contradiction,183187,183185
entailment,183416,183414
neutral,182764,182762


In [ ]:
df_test.groupby('gold_label').count()

,sentence1,sentence2
gold_label,,
contradiction,3237,3237
entailment,3368,3368
neutral,3219,3219


In [ ]:
df_dev.groupby('gold_label').count()

,sentence1,sentence2
gold_label,,
contradiction,3278,3278
entailment,3329,3329
neutral,3235,3235


In [ ]:
#  dropping the rows from the data with NaN values from df_train

df_train = df_train.dropna()


In [ ]:
print(len(df_train), len(df_test), len(df_dev))

549361 9824 9842


In [ ]:
for i, row in df_train.iterrows():
    df_train.at[i, 'input'] = " <cls> " + str(row[1]) + " <sep> " + str(row[2]) + " <cls> "
    if row[0] == 'neutral':
        df_train.at[i, 'label'] = int(0)
    elif row[0] == 'entailment':
        df_train.at[i, 'label'] = int(1)
    elif row[0] == 'contradiction':
        df_train.at[i, 'label'] = int(2)
    else:
        print(str(row[0]))
        break

    if(i % 100000) == 0 and i:
        print("Completed: %s" %(i))

/tmp/ipykernel_16000/2173431417.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.at[i, 'input'] = " <cls> " + str(row[1]) + " <sep> " + str(row[2]) + " <cls> "
/tmp/ipykernel_16000/2173431417.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.at[i, 'label'] = int(0)


Completed: 100000
Completed: 200000
Completed: 300000
Completed: 400000
Completed: 500000


In [ ]:
for i, row in df_dev.iterrows():
    df_dev.at[i, 'input'] = " <cls> " + str(row[1]) + " <sep> " + str(row[2]) + " <cls> "

    if row[0] == 'neutral':
        df_dev.at[i, 'label'] = int(0)
    elif row[0] == 'entailment':
        df_dev.at[i, 'label'] = int(1)
    elif row[0] == 'contradiction':
        df_dev.at[i, 'label'] = int(2)
    else:
        print(row[0])
        break

    if(i % 100000) == 0 and i:
        print("Completed: %s" %(i))

In [ ]:
for i, row in df_test.iterrows():
    df_test.at[i, 'input'] = " <cls> " + str(row[1]) + " <sep> " + str(row[2]) + " <cls> "

    if row[0] == 'neutral':
        df_test.at[i, 'label'] = int(0)
    elif row[0] == 'entailment':
        df_test.at[i, 'label'] = int(1)
    elif row[0] == 'contradiction':
        df_test.at[i, 'label'] = int(2)
    else:
        print(row[0])
        break

    if(i % 100000) == 0 and i:
        print("Completed: %s" %(i))

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:

df_train.head()

,gold_label,sentence1,sentence2,input,label
0,neutral,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,<cls> A person on a horse jumps over a broken...,0.0
1,contradiction,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",<cls> A person on a horse jumps over a broken...,2.0
2,entailment,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",<cls> A person on a horse jumps over a broken...,1.0
3,neutral,Children smiling and waving at camera,They are smiling at their parents,<cls> Children smiling and waving at camera <...,0.0
4,entailment,Children smiling and waving at camera,There are children present,<cls> Children smiling and waving at camera <...,1.0


In [ ]:
##Convert the dataset into a list

##train
inp_train = df_train['input'].tolist()
labels_train = df_train['label'].astype(int).tolist()

##test
inp_test = df_test['input'].tolist()
labels_test = df_test['label'].astype(int).tolist()

##dev
inp_dev = df_dev['input'].tolist()
labels_dev = df_dev['label'].astype(int).tolist()

In [ ]:
inp_train[:10]

[' <cls> A person on a horse jumps over a broken down airplane. <sep> A person is training his horse for a competition. <cls> ',
 ' <cls> A person on a horse jumps over a broken down airplane. <sep> A person is at a diner, ordering an omelette. <cls> ',
 ' <cls> A person on a horse jumps over a broken down airplane. <sep> A person is outdoors, on a horse. <cls> ',
 ' <cls> Children smiling and waving at camera <sep> They are smiling at their parents <cls> ',
 ' <cls> Children smiling and waving at camera <sep> There are children present <cls> ',
 ' <cls> Children smiling and waving at camera <sep> The kids are frowning <cls> ',
 ' <cls> A boy is jumping on skateboard in the middle of a red bridge. <sep> The boy skates down the sidewalk. <cls> ',
 ' <cls> A boy is jumping on skateboard in the middle of a red bridge. <sep> The boy does a skateboarding trick. <cls> ',
 ' <cls> A boy is jumping on skateboard in the middle of a red bridge. <sep> The boy is wearing safety equipment. <cls> ',

In [ ]:
labels_train[:10]

[0, 2, 1, 0, 1, 2, 2, 1, 0, 0]

In [ ]:
lr = 2e-5
num_epochs = 3
MAX_LEN = 128
batch_size = 128
dataset = 'snli'


# In[4]:


ending_path = ('%s_%s_%s_%s' %(dataset, MAX_LEN, batch_size, str(lr).replace("-", "")))


In [ ]:
ending_path

'snli_128_128_2e05'

In [ ]:
def create_attention_masks(input_ids):
    """Creates attention masks from input IDs."""
    attention_masks = []
    for seq in input_ids:
        seq_mask = [float(i > 0) for i in seq]
        attention_masks.append(seq_mask)
    return attention_masks

# TRAIN

In [ ]:
tokenized_texts_train = [tokenizer.tokenize(inp) for inp in inp_train]
input_ids_train = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_train]


In [ ]:

input_ids_train = pad_sequence([torch.tensor(x) for x in input_ids_train], batch_first=True, padding_value=0)
attention_masks_train = create_attention_masks(input_ids_train)

KeyboardInterrupt: 

In [ ]:
print(len(input_ids_train), len(attention_masks_train))

49947 49947


In [ ]:
train_inputs = torch.tensor(input_ids_train)
train_labels = torch.tensor(labels_train)
train_masks = torch.tensor(attention_masks_train)

<ipython-input-36-5767c1a4327c>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_inputs = torch.tensor(input_ids_train)


In [ ]:
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# DEV

In [ ]:
tokenized_texts_dev = [tokenizer.tokenize(inp) for inp in inp_dev]
input_ids_dev = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_dev]

In [ ]:

input_ids_dev = pad_sequence([torch.tensor(x) for x in input_ids_dev], batch_first=True, padding_value=0)
attention_masks_dev = create_attention_masks(input_ids_dev)

In [ ]:
print(len(input_ids_dev), len(attention_masks_dev))

9842 9842


In [ ]:
dev_inputs = torch.tensor(input_ids_dev)
dev_labels = torch.tensor(labels_dev)
dev_masks = torch.tensor(attention_masks_dev)

<ipython-input-41-f7874fc6c6a4>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dev_inputs = torch.tensor(input_ids_dev)


In [ ]:
dev_data = TensorDataset(dev_inputs, dev_masks, dev_labels)
dev_sampler = RandomSampler(dev_data)
dev_dataloader = DataLoader(dev_data, sampler=dev_sampler, batch_size=batch_size)


# TEST

In [ ]:
tokenized_texts_test = [tokenizer.tokenize(inp) for inp in inp_test]
input_ids_test = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_test]

In [ ]:

input_ids_test = pad_sequence([torch.tensor(x) for x in input_ids_test], batch_first=True, padding_value=0)
attention_masks_test = create_attention_masks(input_ids_test)

In [ ]:
print(len(input_ids_test), len(attention_masks_test))

9824 9824


In [ ]:
test_inputs = torch.tensor(input_ids_test)
test_labels = torch.tensor(labels_test)
test_masks = torch.tensor(attention_masks_test)

<ipython-input-46-3b755e796fa7>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_inputs = torch.tensor(input_ids_test)


In [ ]:
test_data = TensorDataset(test_inputs, test_masks, test_labels)
#test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

## Model

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

model = nn.DataParallel(model)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DataParallel(
  (module): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, 

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=lr)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    labels_flat = labels_flat.cpu().detach().numpy()
    return np.sum(pred_flat == labels_flat), pred_flat

In [ ]:
def train(i):
    model.train()
    total_loss = 0.0
    total_predicted_label = np.array([])
    total_actual_label = np.array([])
    train_len = 0
    f_acc = 0

    ## adaptive lr
    optimizer.param_groups[0]['lr'] *= (0.1)**(1/40.)


    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        if b_labels.size(0) == 1:
            continue
        optimizer.zero_grad()
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        pred = outputs[1].detach().cpu().numpy()
        batch_f_acc, pred_flat = flat_accuracy(pred, b_labels)
        f_acc += batch_f_acc
        loss = outputs[0]
        loss.sum().backward()
        optimizer.step()


        labels_flat = b_labels.flatten().cpu().detach().numpy()
        total_actual_label = np.concatenate((total_actual_label, labels_flat))
        total_predicted_label = np.concatenate((total_predicted_label, pred_flat))

        total_loss += outputs[0].sum()
        train_len += b_input_ids.size(0)

        if step%100 == 0 and step:
            precision, recall, f1_measure, _ =             precision_recall_fscore_support(total_actual_label, total_predicted_label, average='macro')
            print("Train: %5.1f\tEpoch: %d\tIter: %d\tLoss: %5.5f\tAcc= %5.3f\tPrecision= %5.3f\tRecall= %5.3f\tF1_score= %5.3f" %                         (train_len*100.0/train_inputs.size(0), i, step,                            total_loss/train_len, f_acc*100.0/train_len,                            precision*100., recall*100., f1_measure*100.))

    #if torch.cuda.device_count() > 1:
        #p = 100
       # path = save_model_path + '/e_' + str(i) + "_" + str(p) + ".ckpt"
        #torch.save(model.module.state_dict(), path)
    #else:
        #torch.save(model.state_dict(), path)
    precision, recall, f1_measure, _ =     precision_recall_fscore_support(total_actual_label, total_predicted_label, average='macro')
    print("Train: %5.1f\tEpoch: %d\tIter: %d\tLoss: %5.5f\tAcc= %5.3f\tPrecision= %5.3f\tRecall= %5.3f\tF1_score= %5.3f" %                 (train_len*100.0/train_inputs.size(0), i, step,                    total_loss/train_len, f_acc*100.0/train_len,                    precision*100., recall*100., f1_measure*100.))
    return total_loss/train_len

In [ ]:
def dev(i):
    model.eval()
    val_len = 0
    total_loss = 0
    total_predicted_label = np.array([])
    total_actual_label = np.array([])
    f_acc = 0

    with torch.no_grad():
        for step, batch in enumerate(dev_dataloader):
            batch = tuple(t.cuda() for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            if b_labels.size(0) == 1:
                continue

            optimizer.zero_grad()
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            pred = outputs[1].detach().cpu().numpy()
            batch_f_acc, pred_flat = flat_accuracy(pred, b_labels)
            f_acc += batch_f_acc

            labels_flat = b_labels.flatten().cpu().detach().numpy()
            total_actual_label = np.concatenate((total_actual_label, labels_flat))
            total_predicted_label = np.concatenate((total_predicted_label, pred_flat))

            val_len += b_input_ids.size(0)
            total_loss += outputs[0].sum()

            if step%100 == 0 and step:
                precision, recall, f1_measure, _ =                 precision_recall_fscore_support(total_actual_label, total_predicted_label, average='macro')
                print("Eval: %5.1f\tEpoch: %d\tIter: %d\tLoss: %5.5f\tAcc= %5.3f\tPrecision= %5.3f\tRecall= %5.3f\tF1_score= %5.3f" %                             (val_len*100.0/dev_inputs.size(0), i, step,                              total_loss/val_len, f_acc*100.0/val_len,                                precision*100., recall*100., f1_measure*100.))

        precision, recall, f1_measure, _ =         precision_recall_fscore_support(total_actual_label, total_predicted_label, average='macro')
        print("Validation: %5.1f\tEpoch: %d\tIter: %d\tLoss: %5.5f\tAcc= %5.3f\tPrecision= %5.3f\tRecall= %5.3f\tF1_score= %5.3f" %                     (val_len*100.0/dev_inputs.size(0), i, step,                      total_loss/val_len, f_acc*100.0/val_len,                        precision*100., recall*100., f1_measure*100.))
    return total_loss/val_len


In [ ]:
def test(i):
    model.eval()
    val_len = 0
    total_loss = 0
    total_predicted_label = np.array([])
    total_actual_label = np.array([])
    f_acc = 0

    with torch.no_grad():
        for step, batch in enumerate(test_dataloader):
            batch = tuple(t.cuda() for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            if b_labels.size(0) == 1:
                continue

            optimizer.zero_grad()
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            pred = outputs[1].detach().cpu().numpy()
            batch_f_acc, pred_flat = flat_accuracy(pred, b_labels)
            f_acc += batch_f_acc

            labels_flat = b_labels.flatten().cpu().detach().numpy()
            total_actual_label = np.concatenate((total_actual_label, labels_flat))
            total_predicted_label = np.concatenate((total_predicted_label, pred_flat))

            val_len += b_input_ids.size(0)
            total_loss += outputs[0].sum()

            if step%100 == 0 and step:
                precision, recall, f1_measure, _ =                 precision_recall_fscore_support(total_actual_label, total_predicted_label, average='macro')
                print("Eval: %5.1f\tEpoch: %d\tIter: %d\tLoss: %5.5f\tAcc= %5.3f\tPrecision= %5.3f\tRecall= %5.3f\tF1_score= %5.3f" %                             (val_len*100.0/test_inputs.size(0), i, step,                              total_loss/val_len, f_acc*100.0/val_len,                                precision*100., recall*100., f1_measure*100.))

        precision, recall, f1_measure, _ =         precision_recall_fscore_support(total_actual_label, total_predicted_label, average='macro')
        print("Test: %5.1f\tEpoch: %d\tIter: %d\tLoss: %5.5f\tAcc= %5.3f\tPrecision= %5.3f\tRecall= %5.3f\tF1_score= %5.3f" %                     (val_len*100.0/test_inputs.size(0), i, step,                      total_loss/val_len, f_acc*100.0/val_len,                        precision*100., recall*100., f1_measure*100.))
        return total_actual_label, total_predicted_label

In [ ]:
train_loss = []
val_loss = []
import pickle as pkl
for i in range(num_epochs):
    train_loss.append(train(i))
    val_loss.append(dev(i))
    actual, predicted = test(0)

    pkl.dump(actual, open('actual', 'wb'))

    pkl.dump(predicted, open('predicted', 'wb'))

pkl.dump(train_loss, open('train_loss.pkl', 'wb'))

pkl.dump(val_loss, open('val_loss.pkl', 'wb'))






LR: 1.888121752571847e-05	Epoch: 0	
(128,) (128,)
(256,) (256,)
(384,) (384,)
(512,) (512,)
(640,) (640,)
(768,) (768,)
(896,) (896,)
(1024,) (1024,)
(1152,) (1152,)
(1280,) (1280,)
(1408,) (1408,)
(1536,) (1536,)
(1664,) (1664,)
(1792,) (1792,)
(1920,) (1920,)
(2048,) (2048,)
(2176,) (2176,)
(2304,) (2304,)
(2432,) (2432,)
(2560,) (2560,)
(2688,) (2688,)
(2816,) (2816,)
(2944,) (2944,)
(3072,) (3072,)
(3200,) (3200,)
(3328,) (3328,)
(3456,) (3456,)
(3584,) (3584,)
(3712,) (3712,)
(3840,) (3840,)
(3968,) (3968,)
(4096,) (4096,)
(4224,) (4224,)
(4352,) (4352,)
(4480,) (4480,)
(4608,) (4608,)
(4736,) (4736,)
(4864,) (4864,)
(4992,) (4992,)
(5120,) (5120,)
(5248,) (5248,)
(5376,) (5376,)
(5504,) (5504,)
(5632,) (5632,)
(5760,) (5760,)
(5888,) (5888,)
(6016,) (6016,)
(6144,) (6144,)
(6272,) (6272,)
(6400,) (6400,)
(6528,) (6528,)
(6656,) (6656,)
(6784,) (6784,)
(6912,) (6912,)
(7040,) (7040,)
(7168,) (7168,)
(7296,) (7296,)
(7424,) (7424,)
(7552,) (7552,)
(7680,) (7680,)
(7808,) (7808,)
(7

TypeError: unsupported format string passed to list.__format__

In [ ]:
# save the model
torch.save(model.state_dict(), 'model_state_dict.pth')
torch.save(model, 'model.pth')